# Amazon SageMaker - Bring Your Own Model 
## MXNet + Gluon 編

ここでは [Apache MXNet](https://mxnet.apache.org/) と [Gluon](https://mxnet.incubator.apache.org/versions/master/gluon/index.html) を使ったサンプルコードを題材に、Amazon SageMaker 移行の方法を順を追って説明します。SageMaker Python SDK で MXNet を使うための説明は [SDK のドキュメント](https://sagemaker.readthedocs.io/en/stable/using_mxnet.html) にも多くの情報があります。

注: 
ここで説明するのは Script モード という記法 (現時点では標準の書き方) で、FILE モード (入力データを Amazon S3 から学習時にファイルとしてコピーする方法) です。データサイズが大きくなった場合は、FILE Mode ではなく PIPE Mode をお使い頂いた方がスループットが向上します。
また、ここでは以降手順の紹介のためトレーニングスクリプトは最小限の書き換えとしています。

## 1. トレーニングスクリプトの書き換え
まず [サンプルのソースコード](https://github.com/apache/incubator-mxnet/blob/master/example/gluon/mnist/mnist.py) を以下のコマンドでダウンロードします。

In [ ]:
!wget https://raw.githubusercontent.com/apache/incubator-mxnet/master/example/gluon/mnist/mnist.py 

ダウンロードされた `mnist.py` を左のファイルブラウザから見つけて、ダブルクリックで開いて下さい (JupyterLab の場合は左右にファイルを並べると作業しやすいです)。あるいはお好きなエディターをお使い頂いても結構です。

書き換える点は主に4点です:
1. 環境変数の取得、
1. 入力データのディレクトリを変更、
1. 学習済みモデルの出力先を変更、
1. 推論のためモデルを読み込む。

その際に `main` 関数の外で呼んでいる部分 (引数のパース、ニューラルネットワークの定義、データ読み込み) をそれぞれ関数として定義します。

### 1-1. 環境変数の取得

Amazon SageMaker の Script Mode では、トレーニングに用いるコードが実行時に Python スクリプトとして実行されます。その際、データ・モデルの入出力は [こちら](https://sagemaker.readthedocs.io/en/stable/using_mxnet.html#preparing-the-mxnet-training-script) に記述があるよう `SM_CHANNEL_XXXX` や `SM_MODEL_DIR` という環境変数を参照する必要があります。そのため、`argparse.ArgumentParser` で渡された環境変数と、スクリプト実行時のハイパーパラメータを取得します。

![データのやりとり](../img/sagemaker-data-model.png)

`SM_CHANNEL_TRAIN`, `SM_CHANNEL_TEST`, `SM_MODEL_DIR` を取得するよう以下のように3行書き足します (`parser.parse_args` を呼んでいる [45行目](https://github.com/apache/incubator-mxnet/blob/master/example/gluon/mnist/mnist.py#L45) の前あたりに)。

```
parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
parser.add_argument('--test', type=str, default=os.environ['SM_CHANNEL_TEST'])
parser.add_argument('--sm-model-dir', type=str, default=os.environ['SM_MODEL_DIR'])

```

その上で、
```
parser = argparse.ArgumentParser(description='MXNet Gluon MNIST Example')
```
から
```
opt = parser.parse_args()
```
までの行を、
`parse_args` などという関数として定義しましょう (`opt` を返します)。途中省略しますがこんな感じです: 

```
def parse_args():

    parser = argparse.ArgumentParser(description='MXNet Gluon MNIST Example')
    parser.add_argument('--batch-size', type=int, default=100,
                        help='batch size for training and testing (default: 100)')
    ...
    parser.add_argument('--sm-model-dir', type=str, default=os.environ['SM_MODEL_DIR'])

    opt = parser.parse_args()
    return opt
```

トレーニングスクリプトは `main` 関数から実行されるので、その冒頭で 
```
opt = parse_args()
```
を呼び出します。`train` 関数の定義で引数を `epochs` ではなく `opt` に書き換えます (`opt.epochs` だけでなく `opt.train`, `opt.test` も使うため)。代わりに train 関数の先頭で
```
epochs = opt.epochs
```
と追記しておきましょう。
また、`main` の中で `train(opt, ctx)` と呼ぶようにします。

### 1-2. 入力データのディレクトリを変更

元のコードでは `gluon.data.DataLoader` を使ってデータのダウンロード・読み込みを行なっています。Amazon SageMaker では先ほど環境変数から取得した `opt.train`, `opt.test` (変数の値は、以下で指定するようにそれぞれ `/opt/ml/input/data/train`, `/opt/ml/input/data/test` となる予定。詳細は [ドキュメント](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo-running-container.html#your-algorithms-training-algo-running-container-trainingdata) をご覧下さい) にデータを読みにいくよう書き換える必要があります。FILE Mode の場合、トレーニングコンテナ起動時に S3 からこれらのディレクトリへデータがコピーされます。

`gluon.data.DataLoader` を使ってデータを読んでいる2箇所の
```
gluon.data.DataLoader(
    gluon.data.vision.MNIST('./data', ...)
```
を、以下のように書き換えます。
```
gluon.data.DataLoader(
    gluon.data.vision.MNIST(opt.train, ...)
gluon.data.DataLoader(
    gluon.data.vision.MNIST(opt.test, ...)
```

元々のコードの場合は、この場所で `gluon.data.vision.MNIST` を使ってインターネットからデータをダウンロードしていました。`gluon.data.vision.MNIST` は指定されたディレクトリにデータを探しに行き、そこにファイルが置かれている場合は別途ダウンロードせずにファイルを直接読み込みむので、この第一引数 `root` を書き換えてあげれば期待通り Amazon S3 からのデータを読んでくれます。

ここも、`load_data` のような関数として定義しましょう (`opt` を引数に取って `train_data, val_data` を返します)。こんな感じです:

```
def load_data(opt):
    train_data = gluon.data.DataLoader(
        gluon.data.vision.MNIST(opt.train, train=True, transform=transformer),
        batch_size=opt.batch_size, shuffle=True, last_batch='discard')

    val_data = gluon.data.DataLoader(
        gluon.data.vision.MNIST(opt.test, train=False, transform=transformer),
        batch_size=opt.batch_size, shuffle=False)
    
    return train_data, val_data
```

`train` 関数の中で以下のように呼び出します。
```
train_data, val_data = load_data(opt)
```
また `test` 関数に `net`, `val_data` を渡すようにします (グローバル変数でなくなったので引数で渡す必要があります)。元のコードの [72行目](https://github.com/apache/incubator-mxnet/blob/master/example/gluon/mnist/mnist.py#L72) と [117行目](https://github.com/apache/incubator-mxnet/blob/master/example/gluon/mnist/mnist.py#L117) です。

後ほど、Notebook 上で予めデータをダウンロードしておいて、Amazon S3 にアップロードする作業を行います。

### 1-3. 学習済みモデルの出力先を変更

モデルは `net.save_parameters('mnist.params')` で保存されています。Amazon SageMaker では、指定されたディレクトリ `opt.sm_model_dir` (すなわち `/opt/ml/model`) に学習結果を保存すると、Amazon S3 へアップロードしてくれます。

以下のように書き換えるやり方でも重みを保存できますが、
> 
> ```net.save_parameters(os.path.join(opt.sm_model_dir, 'mnist.params'))```
> 
> ここで、`os.path.join` を呼んでいるので忘れずに `import os` も書いておきましょう。
> 
> ニューラルネットワークの定義を `net` を返す関数にします。例えば `define_network` という関数名だとすると、`train` 関数の冒頭で `net = define_network()` を呼びます。

ここでは Gluon の [`mxnet.gluon.HybridBlock.export`](https://mxnet.incubator.apache.org/api/python/gluon/gluon.html#mxnet.gluon.HybridBlock.export) を使ってモデルと重みを保存することにします。まず、モデルの定義で `HybridSequential` を使うように
```
net = nn.HybridSequential()
with net.name_scope():
    net.add(nn.Dense(128, activation='relu'))
    net.add(nn.Dense(64, activation='relu'))
    net.add(nn.Dense(10))
net.hybridize()
```
と書き換えます。最後の行で `hybridize` を呼ぶことを忘れないでください。これはモデルのシリアライズのために必要で、パフォーマンスの向上にも寄与します。

`net.save_parameters` を以下に書き換えて、
```
net.export(os.path.join(opt.sm_model_dir, 'mlp'))
```
冒頭 [20行目](https://github.com/apache/incubator-mxnet/blob/master/example/gluon/mnist/mnist.py#L20) あたり に `import os` も書き足します。


### 1-4. 推論のためモデルを読み込む

ここまでの変更で Amazon SageMaker 上のトレーニングが実行できるようになりましたが、モデルのホスティングのため、`model_fn` を定義する必要があります。この関数は `model_dir` を引数にとり、ニューラルネットワーク `net` (ここでは `mxnet.gluon.nn.Sequential`) のモデル (`mlp-symbol.json`) と重み (`mlp-0000.params`) を読み込み `net` を返します。

```
def model_fn(model_dir):
    """
    Load the gluon model. Called once when hosting service starts.

    :param: model_dir The directory where model files are stored.
    :return: a model (in this case a Gluon network)
    """
    
    net = gluon.nn.SymbolBlock.imports('mlp-symbol.json', ['data'], 'mlp-0000.params')
    
    return net
```

前処理・後処理などの詳細は [API ドキュメント](https://sagemaker.readthedocs.io/en/stable/using_mxnet.html#the-sagemaker-mxnet-model-server) に記載があります。

## 2. Notebook 上でのデータ準備

トレーニングスクリプトの書き換えは終了しました。トレーニングを始める前に、予め Amazon S3 にデータを準備しておく必要があります。この Notebook を使ってその作業をします。

In [ ]:
import os
import numpy as np
import boto3
import sagemaker
from sagemaker.mxnet import MXNet
from mxnet import gluon
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

先ほど書き換えたトレーニングスクリプトの中には、`transformer` という関数が定義され前処理のデータ変換 (正規化) が含まれていました。それも含め、`gluon.data.vision.MNIST` でダウンロード・データ変換を行います。

In [ ]:
def transformer(data, label):
    data = data.reshape((-1,)).astype(np.float32)/255
    return data, label

train = gluon.data.vision.MNIST('./data/train', train=True, transform=transformer)
test = gluon.data.vision.MNIST('./data/test', train=False, transform=transformer)

これを Amazon S3 にアップロードします。

In [ ]:
train_data = sagemaker_session.upload_data(path='data/train', key_prefix='data/handson-byom-mxnet-mnist/train')
test_data = sagemaker_session.upload_data(path='data/test', key_prefix='data/handson-byom-mxnet-mnist/test')

## 3. Local Mode によるトレーニングとコードの検証
トレーニングジョブを始める前に、Local Mode を使って、この Notebook インスタンス上でコンテナを立てコードをデバッグしましょう。

`from sagemaker.mxnet import MXNet` で読み込んだ SageMaker Python SDK の MXNet Estimator を作ります。

In [ ]:
train_instance_type = "local"

estimator = MXNet("mnist.py",
                  role=role,
                  train_instance_count=1,
                  train_instance_type=train_instance_type,
                  py_version='py3', 
                  framework_version="1.4.0",
                  hyperparameters={'batch-size': 100,
                                   'epochs': 4,
                                   'lr': 0.1,
                                   'momentum': 0.9, 
                                   'log-interval': 100})

`estimator.fit` によりトレーニングを開始しますが、ここで指定する「チャネル」によって、環境変数名 `SM_CHANNEL_XXXX` が決定されます。この例の場合、`'train', 'test'` を指定しているので、`SM_CHANNEL_TRAIN`, `SM_CHANNEL_TEST` となります。

In [ ]:
estimator.fit({'train': train_data, 'test': test_data})

`mnist.py` の中で書き換えを忘れた部分があったら、ここでエラーとなる場合があります。Local Mode ではクイックにデバッグができるので、正しく実行できるよう試行錯誤しましょう。

 `===== Job Complete =====`
と表示されれば成功です。

### 学習済みモデルの確認

Amazon S3 に保存されたモデルは普通にダウンロードしてきて使うこともできます。保存先は `estimator.model_data` で確認できます。

In [ ]:
estimator.model_data

AWS CLI を使ってノートブックインスタンス上に持ってきて、試しに推論させてみます。

In [ ]:
!aws s3 cp $estimator.model_data ./
!tar -zxvf model.tar.gz 

In [ ]:
from mxnet.gluon import nn

net = gluon.nn.SymbolBlock.imports('mlp-symbol.json', ['data'], 'mlp-0000.params')

In [ ]:
test_dataloader = gluon.data.DataLoader(test, batch_size=10)

for data, label in test_dataloader:
    print('label:', label.asnumpy())
    pred = net(data)
    print('pred: ', pred.asnumpy().argmax(axis=1))
    break

## 4. トレーニングジョブの発行

正しく推論されていればコードのデバッグは完了です。次に、Amazon SageMaker のトレーニングジョブとしてトレーニングをさせます。データ・モデルの入出力は変わらず S3 なので、`train_instance_type` に `ml.` で始まる SageMaker のインスタンスを指定するだけで実行できます。(リストは[こちら](https://aws.amazon.com/sagemaker/pricing/instance-types/))

In [ ]:
train_instance_type = "ml.m4.xlarge"
# train_instance_type = "ml.c5.xlarge"

estimator = MXNet("mnist.py",
                  role=role,
                  train_instance_count=1,
                  train_instance_type=train_instance_type,
                  py_version='py3', 
                  framework_version="1.4.0",
                  hyperparameters={'batch-size': 100,
                                   'epochs': 20,
                                   'lr': 0.1,
                                   'momentum': 0.9, 
                                   'log-interval': 100})

In [ ]:
estimator.fit({'train': train_data, 'test': test_data})

----
```
Billable seconds: <time>
```
と出力されればトレーニング終了です。これが実際にトレーニングインスタンスが課金される時間となります。

## 5. 推論エンドポイントのデプロイ

`estimator.deploy` で、今トレーニングしたモデルを推論エンドポイントとしてデプロイすることができます。これには数分かかります。(`----!` と表示されればデプロイ完了です。)

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [ ]:
test_dataloader = gluon.data.DataLoader(test, batch_size=10)

In [ ]:
for data, label in test_dataloader:
    print('label:', label.asnumpy())
    pred = predictor.predict(data.asnumpy())
    print('pred: ', np.array(pred).argmax(axis=1))
    break

推論エンドポイントは立てっぱなしにしているとお金がかかるので、確認が終わったら忘れないうちに削除してください。

In [ ]:
predictor.delete_endpoint()

## 6. まとめ

Apache MXNet と Gluon を使った Amazon SageMaker への移行手順について紹介しました。普段お使いのモデルでも同様の手順で移行が可能ですのでぜひ試してみてください。